In [203]:
from simulate import BivariateBicycle, SurfaceCode, Algorithm, LogicalCircuit, NoiseModel, CodeFactory
import pandas as pd
import stim
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np

alg = Algorithm.build_memory(cycles=3)
cf = CodeFactory(BivariateBicycle, {'l': 6, 'm': 6})
# cf = CodeFactory(SurfaceCode, {'d': 3})
lc, _, mt = LogicalCircuit.from_algorithm(alg, cf)
noise_model = NoiseModel.get_scaled_noise_model(1.0).without_loss()

tanner = pd.DataFrame(cf().metadata.tanner)
c2c = tanner.merge(tanner, on='data_id')[['check_id_x', 'check_id_y', 'edge_type_x', 'edge_type_y']]
all_edge_types = []
graph = []

for (c1,c2), df in c2c.groupby(['check_id_x', 'check_id_y']):
    edge_types = df[['edge_type_x', 'edge_type_y']].to_numpy()
    edge_types = frozenset([tuple(map(int, x)) for x in edge_types])
    if edge_types not in all_edge_types:
        all_edge_types.append(edge_types)
    c1, c2 = map(int, (c1, c2))
    graph.append((c1, c2, all_edge_types.index(edge_types)))

graph = pd.DataFrame(graph, columns=['syndrome_id', 'neighbor_syndrome_id', 'edge_type'])

In [216]:
det = pd.DataFrame(mt.detectors)
det_graph = det.merge(graph, on='syndrome_id').merge(det, left_on='neighbor_syndrome_id', right_on='syndrome_id', suffixes=('', '_nb'))
det_graph = det_graph[np.abs(det_graph['time'] - det_graph['time_nb']) <= 1]
dt = det_graph['time_nb'] - det_graph['time'] + 1
det_graph['edge_type'] = dt * len(all_edge_types) + det_graph['edge_type']
det_graph = det_graph[['detector_id', 'detector_id_nb', 'edge_type']]
det_graph = det_graph.sort_values(by=['edge_type', 'detector_id', 'detector_id_nb'])
det_graph

,detector_id,detector_id_nb,edge_type
5544,72,0,0
5622,73,1,0
5697,74,2,0
5772,75,3,0
5847,76,4,0
...,...,...,...
16303,211,244,131
16382,212,245,131
16457,213,240,131
16536,214,241,131
